In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.font_manager as fm
import glob



In [3]:
# 푸드 데이터 로드
path_food = "./data/EC_SNS_KFOOD_ATTRACTION_DATA_*.csv"
files_food = glob.glob(path_food)
df_food = [pd.read_csv(f, encoding='utf-8') for f in files_food]
sns_kfood = pd.concat(df_food, ignore_index=True)

# 뷰티 데이터 로드
path_beauty = "./data/EC_SNS_KBEAUTY_ATTRACTION_DATA_*.csv"
files_beauty = glob.glob(path_beauty)
df_beauty = [pd.read_csv(f, encoding='utf-8') for f in files_beauty]
sns_beauty = pd.concat(df_beauty, ignore_index=True)

# 데이터 병합
full_df = pd.concat([sns_kfood, sns_beauty], ignore_index=True)

### 필요한 컬럼명 정리
#### 그대로 readme에 올려도 될듯합니다
|컬럼영문명|컬럼한글명|필요 컬럼|
|---|---|---|
|TRRSRT_NM|관광지명||
|BASE_YM|기준년월|O|
|CHNNEL_NM|채널명||
|BASE_YEAR_ACCMLT_FQ_CO|기준년도누적빈도수||
|BASE_YM_FQ_CO|	기준년월빈도수||
|BASE_YEAR_BEFORE_MT_FQ_CO|기준년도이전월빈도수||
|BEFORE_MT_VERSUS_FQ_CO_IRDS_RT|이전월대비빈도수증감율||
|AVRG_SCORE_VALUE|평점값|O|
|REVIEW_CO|리뷰수|O|
|PLACE_TY|장소유형||
|LC_LA|위치위도||
|LC_LO|위치경도||
|ADDR|주소||
|CTPRVN_NM|시도명|O|
|SIGNGU_NM|시군구명|O|
|TURSM_CSTMR_CO|관광고객수|O|
|BEFORE_YEAR_MT_TURSM_CSTMR_CO|이전년도월관광고객수||
|TURSM_CSTMR_CO_IRDS_RT|관광고객수증감율||
|TURSM_SPND_PRICE|관광소비금액|O|
|NATIVE_TURSM_SPND_PRICE|내국인관광소비금액|O|
|FRNR_TURSM_SPND_PRICE|외국인관광소비금액|O|

In [4]:
# 채널 이름이 체널 전체인 것들만 추출 후 0으로 초기화
full_df = full_df[full_df['CHNNEL_NM'] == '채널전체'].copy()
full_df = full_df.fillna(0)

# 사용할 컬럼 추출
use_cols = ['BASE_YM', 'CTPRVN_NM', 'SIGNGU_NM', 'TURSM_CSTMR_CO', 
            'TURSM_SPND_PRICE', 'NATIVE_TURSM_SPND_PRICE', 
            'FRNR_TURSM_SPND_PRICE', 'AVRG_SCORE_VALUE', 'REVIEW_CO', 
            'BASE_YM_FQ_CO', 'BASE_YEAR_BEFORE_MT_FQ_CO'] 
df_sub = full_df[use_cols].copy()

# 데이터 형 변환
df_sub['BASE_YM'] = pd.to_datetime(df_sub['BASE_YM'], format='%Y%m')
df_sub = df_sub.fillna(0)

# 통합 데이터 셋 (뷰티는 구별로 다른 구간도 있고 같은 구간도 있어서 평균으로 처리함)    
final_df = df_sub.groupby(['BASE_YM', 'CTPRVN_NM', 'SIGNGU_NM']).agg({
    'TURSM_SPND_PRICE': 'mean',          # 관광소비금액
    'TURSM_CSTMR_CO': 'mean',            # 관광고객수
    'NATIVE_TURSM_SPND_PRICE': 'mean',   # 내국인관광소비금액
    'FRNR_TURSM_SPND_PRICE': 'mean',     # 외국인관광소비금액
    'AVRG_SCORE_VALUE': 'mean',          # 평점값
    'REVIEW_CO': 'mean',                 # 리뷰수
    'BASE_YM_FQ_CO': 'sum',              # 기준년월빈도수
    'BASE_YEAR_BEFORE_MT_FQ_CO' : 'sum'  # SNS기준년도이전우러빈도수
}).reset_index()

# SNS 한 달 간의 증감량
final_df['SNS_MONTHLY_DIFF'] = final_df['BASE_YM_FQ_CO'] - final_df['BASE_YEAR_BEFORE_MT_FQ_CO']


In [5]:

# pct_change(): 전년대비 변화량을 계산해 주는 함수 <(현재값 - 이전값) / 이전값>
# diff() : 이전값 대비 현재값의 변화량을 계산해 주는 함수 <현재값 - 이전값>
# (현재값 - 이전값) / 이전값 * 100 = 현재 성장률
final_df = final_df.sort_values(['SIGNGU_NM', 'BASE_YM'])   # pct_change()를 쓰려면 정렬이 필요
final_df = final_df.reset_index(drop=True)

# 몇몇개의 시군구명이 KR로 들어가 있는 데이터랑 '0' 들어가 있는 데이터 제거
final_df = final_df[~final_df['SIGNGU_NM'].isin(['KR', '0', 0])]
final_df = final_df.reset_index(drop=True)

# 전월 대비 소비 증감률 = (현재달 - 지난달) / 지난달 * 100
# final_df['SPND_GROWTH_RATE'] = final_df.groupby('SIGNGU_NM')['TURSM_SPND_PRICE'].pct_change() * 100
# 전월 대비 소비 증감액 = 현재달 - 지난달 (단위: 백만원)
final_df['SPND_DIFF_VALUE'] = final_df.groupby('SIGNGU_NM')['TURSM_SPND_PRICE'].diff() / 1000000

# 관광고객수 증감률 / 증감액
# final_df['CSTMR_GROWTH_RATE'] = final_df.groupby('SIGNGU_NM')['TURSM_CSTMR_CO'].pct_change() * 100
final_df['CSTMR_DIFF_VALUE'] = final_df.groupby('SIGNGU_NM')['TURSM_CSTMR_CO'].diff()

# 리뷰수 증감률 / 증감수
# final_df['REVIEW_GROWTH_RATE'] = final_df.groupby('SIGNGU_NM')['REVIEW_CO'].pct_change() * 100
final_df['REVIEW_DIFF_VALUE'] = final_df.groupby('SIGNGU_NM')['REVIEW_CO'].diff()

# 검색빈도수 증감수
final_df['SNS_DIFF_VALUE'] = final_df.groupby('SIGNGU_NM')['BASE_YEAR_BEFORE_MT_FQ_CO'].diff()   

# 결측치 및 무한대를 이전 값으로 처리 
# 변화률 위주이기 때문에 0을 넣어서 이전달과 변화가 없다 라고 평가하는게 맞다고 봄
final_df = final_df.replace([np.inf, -np.inf], np.nan)
final_df = final_df.fillna(0)

# 이상치 제거
# 중간에 보면 증감률이 8000 까지 뛸때가 있음
# 실측 오류이거나 데이터 등록에서 오류가 발생했다고 보고 마찬가지로 변화가 없다는 의미의 0으로 처리
# final_df.loc[abs(final_df['SPND_GROWTH_RATE']) > 500, ['SPND_GROWTH_RATE', 'SPND_DIFF_VAL_M']] = 0
# final_df.loc[abs(final_df['CSTMR_GROWTH_RATE']) > 500, ['CSTMR_GROWTH_RATE', 'CSTMR_DIFF_VALUE']] = 0
# final_df.loc[abs(final_df['REVIEW_GROWTH_RATE']) > 500, ['REVIEW_GROWTH_RATE', 'REVIEW_DIFF_VALUE']] = 0

# 증감률 즉 퍼센테이지는 당장에 필요 없을꺼같아서 우선 주석처리 해봤습니다.

final_df

,BASE_YM,CTPRVN_NM,SIGNGU_NM,TURSM_SPND_PRICE,TURSM_CSTMR_CO,NATIVE_TURSM_SPND_PRICE,FRNR_TURSM_SPND_PRICE,AVRG_SCORE_VALUE,REVIEW_CO,BASE_YM_FQ_CO,BASE_YEAR_BEFORE_MT_FQ_CO,SNS_MONTHLY_DIFF,SPND_DIFF_VALUE,CSTMR_DIFF_VALUE,REVIEW_DIFF_VALUE,SNS_DIFF_VALUE
0,2025-02-01,경기도,가평군,4526283.0,1330686.0,4356036.0,170247.0,4.4,36432.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
1,2025-07-01,경기도,가평군,655573.0,2688308.0,285133.0,370440.0,4.4,36800.000000,1.0,0.0,1.0,-3.870710,1357622.0,368.000000,0.0
2,2025-08-01,경기도,가평군,56729443.0,3322011.0,56258056.0,471387.0,4.4,36877.000000,1.0,1.0,0.0,56.073870,633703.0,77.000000,1.0
3,2025-09-01,경기도,가평군,39090303.0,2051913.0,38434403.0,655900.0,4.4,30306.666667,2.0,1.0,1.0,-17.639140,-1270098.0,-6570.333333,0.0
4,2025-10-01,경기도,가평군,48143693.0,2644120.0,47364911.0,778782.0,4.4,30416.666667,0.0,2.0,-2.0,9.053390,592207.0,110.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851,2025-11-01,강원특별자치도,홍천군,33648648.0,1640793.0,33445264.0,203384.0,4.3,6697.000000,0.0,0.0,0.0,-9.659481,-606483.0,2.000000,0.0
852,2025-12-01,강원특별자치도,홍천군,36272116.0,1632712.0,34719603.0,1552513.0,4.3,6705.000000,0.0,0.0,0.0,2.623468,-8081.0,8.000000,0.0
853,2025-03-01,서울특별시,화양동,37341094.0,6797980.0,22373501.0,14967593.0,4.1,812.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
854,2025-04-01,서울특별시,화양동,33977729.0,5887682.0,21736852.0,12240877.0,4.1,817.000000,0.0,0.0,0.0,-3.363365,-910298.0,5.000000,0.0


## ML

### 내국인 관광소비금액 예측
예측 타깃
내국인 관광 소비금액
내국인 관광 소비 예측은 정책·마케팅 활용성 명확
1 수요 관련

TURSM_CSTMR_CO

NATIVE_TURSM_SPND_PRICE

이전월 내국인 소비금액 (lag)

2 지역

CTPRVN_NM

SIGNGU_NM

3 인지도/매력

AVRG_SCORE_VALUE

REVIEW_CO

REVIEW_GROWTH_RATE

4 시계열

월 (BASE_YM → month)

성수기 더미

모델
Linear Regression (baseline)

RandomForestRegressor

GradientBoosting / XGBoost

평가
RMSE / MAE

지역별 오차 비교

In [6]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error, r2_score
from sklearn.linear_model import Ridge
from sklearn.impute import SimpleImputer

pd.options.display.float_format = '{:.3f}'.format # 소수점 3자리까지만 표시

In [7]:
df = final_df.copy() # 복사본
# 날짜 처리
df['BASE_YM'] = pd.to_datetime(df['BASE_YM'])

# 지역 키
group_keys = ['CTPRVN_NM', 'SIGNGU_NM']

# 월 파생
df["year"] = df["BASE_YM"].dt.year
df["month"] = df["BASE_YM"].dt.month
df["quarter"] = df["BASE_YM"].dt.quarter

# 나눗셈 방지
eps = 1e-9
df['native_share_spend'] = df['NATIVE_TURSM_SPND_PRICE'] / (df['TURSM_SPND_PRICE'] + eps)
df['native_share_spend'] = df['native_share_spend'].clip(0, 1)

# 이동평균 및 이동표준편차 피처 생성
for col in ["NATIVE_TURSM_SPND_PRICE", "TURSM_CSTMR_CO", "REVIEW_CO"]:
    g = df.groupby(group_keys)[col]
    df[f"{col}_roll3_mean"] = g.shift(1).rolling(3, min_periods=1).mean()
    df[f"{col}_roll3_std"]  = g.shift(1).rolling(3, min_periods=2).std()


# 타겟 변수

# 학습에 쓰지 않을 컬럼
drop_cols = ['NATIVE_TURSM_SPND_PRICE']

# 카테고리/수치 피처 구분
categorical_features = ["CTPRVN_NM", "SIGNGU_NM"]  # 지역
exclude_cols = ["native_share_spend"] # 누수 변수


numeric_features = [c for c in df.columns
                    if c not in (drop_cols + exclude_cols + ["BASE_YM", "CTPRVN_NM", "SIGNGU_NM"])]

X = df[categorical_features + numeric_features]
y = df['NATIVE_TURSM_SPND_PRICE'].clip(0, None)  # 음수 방지

# 테스트 데이터 분리
# train_test_split 안쓴이유는 시계열 데이터이기 때문 -> 시간기반
# last_date = df["BASE_YM"].max()
# test_start = (last_date - pd.offsets.MonthBegin(3))  # 대략 마지막 3개월
# train_idx = df["BASE_YM"] < test_start
# test_idx  = df["BASE_YM"] >= test_start

# X_train, y_train = X.loc[train_idx], y.loc[train_idx]
# X_test,  y_test  = X.loc[test_idx],  y.loc[test_idx]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, shuffle=False)
# 전처리 파이프라인

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='drop'
)

In [8]:
# 모델 선택

ridge = Ridge(alpha=1.0, random_state=42)

# 트리 기반 
hgb = HistGradientBoostingRegressor(
    learning_rate=0.05,
    max_depth=5,
    max_iter=500,
    random_state=42
)

# 랜덤 포레스트

rf_clf = RandomForestRegressor(
    n_estimators=400,
    max_depth=None,
    min_samples_leaf=5,
    n_jobs=-1,
    random_state=42
)

model = {
    'Ridge' : ridge,
    'HistGB' : hgb,
    'RF' : rf_clf
}

In [9]:
def eval(m_n, model, X_train, y_train, X_test, y_test):
    # 파이프라인 생성
    pipe = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    pipe.fit(X_train, y_train)
    
    y_pred = pipe.predict(X_test)
    
    pred = np.clip(y_pred, 0, None)
    true = y_test.to_numpy()
    
    mae = mean_absolute_error(true, pred)
    rmse = root_mean_squared_error(true, pred)
    
    print(f"{m_n} - MAE: {mae:,.0f}, RMSE: {rmse:,.0f}")
    return pipe, {'MAE': mae, 'RMSE': rmse}

fitted_models = {}
scores = {}

for m_n, m in model.items():
    pipe, score = eval(m_n, m, X_train, y_train, X_test, y_test)
    fitted_models[m_n] = pipe
    scores[m_n] = score

print(f'모델 평가 결과 {scores}')


Ridge - MAE: 34,609,860, RMSE: 55,715,383
HistGB - MAE: 17,581,868, RMSE: 53,412,951
RF - MAE: 21,033,971, RMSE: 66,743,226
모델 평가 결과 {'Ridge': {'MAE': 34609860.06989768, 'RMSE': 55715382.62363712}, 'HistGB': {'MAE': 17581868.353904262, 'RMSE': 53412951.363147005}, 'RF': {'MAE': 21033970.983724914, 'RMSE': 66743226.293223865}}


In [11]:
# 지역별 예측 결과 샘플 확인
sample_name = pd.DataFrame(scores).T.sort_values('RMSE').index[0]
best_model = fitted_models[sample_name]

test_out = df.loc[X_test.index, ["BASE_YM", "CTPRVN_NM", "SIGNGU_NM", "NATIVE_TURSM_SPND_PRICE"]].copy()
pred_amt = best_model.predict(X_test)
test_out["PRED_NATIVE_TURSM_SPND_PRICE"] = np.clip(pred_amt, 0, None)  # 원 스케일
display(test_out.sort_values('PRED_NATIVE_TURSM_SPND_PRICE', ascending=False).head(50))
# display(test_out[test_out['NATIVE_TURSM_SPND_PRICE'] < test_out['PRED_NATIVE_TURSM_SPND_PRICE']].sort_values('PRED_NATIVE_TURSM_SPND_PRICE', ascending=False).head(50))

,BASE_YM,CTPRVN_NM,SIGNGU_NM,NATIVE_TURSM_SPND_PRICE,PRED_NATIVE_TURSM_SPND_PRICE
711,2025-11-01,서울특별시,중구,636964129.000,400316838.857
715,2025-12-01,서울특별시,중구,701404729.000,382493166.972
703,2025-09-01,서울특별시,중구,612421734.000,370829330.058
707,2025-10-01,서울특별시,중구,619078971.000,370467325.841
716,2025-12-01,인천,중구,205245440.000,285121009.177
717,2025-12-01,인천광역시,중구,205245440.000,276397120.758
712,2025-11-01,인천광역시,중구,195622209.000,274184929.062
704,2025-09-01,인천광역시,중구,199860666.000,273890135.061
708,2025-10-01,인천광역시,중구,236000807.000,272211315.286
699,2025-08-01,서울특별시,중구,614363605.000,226783314.575
